# EMT Simulation of topology with slack, line and VSI

In [6]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import numpy as np
import matplotlib.pyplot as plt
import re
import dpsimpy

#%matplotlib widget

In [7]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/dpsim/examples/cxx

EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init

[12:01:19.852564 PiLine warning] Zero value for Capacitance, setting default value of C=1e-12 [F]
[12:01:19.852595 PiLine warning] Zero value for Conductance, setting default value of G=1e-06 [S]
[12:01:19.864776 EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init_PF info] Initialize simulation: EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init_PF
[12:01:19.872655 EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init_PF_PF_Solver warning] No suitable quantity found for setting mBaseApparentPower. Using 100000000 VA.
[12:01:19.880516 EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init_PF info] Scheduling tasks.
[12:01:19.885702 EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init_PF info] Scheduling done.
[12:01:19.885718 EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init_PF info] Opening interfaces.
[12:01:19.885719 EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init_PF info] Start synchronization with 

In [5]:
modelName = 'EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init_EMT'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'
PEAK1PH_TO_RMS3PH = np.sqrt(3.0/2.0)
ts_dpsim = read_timeseries_csv(dpsim_result_file)
# ts_dpsim_simulink = read_timeseries_csv('logs/pcc_transient_simulink.csv')
# ts_dpsim_v =  read_timeseries_csv('logs/voltage_pcc_simulink.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'logs/EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init_EMT/EMT_Slack_PiLine_VSI_VoltageControlled_SteadyState_with_PF_Init_EMT.csv'

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Spannung PCC - DPsim')
plt.xlabel('Zeit t [ms]')
plt.ylabel('Spannung U [V]')
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name in ['Spannung_PCC_0', 'Spannung_PCC_1', 'Spannung_PCC_2']:
        plt.plot(1000*ts_obj.time, ts_obj.values, label=ts_name)
plt.axhline(y =400, color = 'r', linestyle = '-')
plt.axhline(y =-400, color = 'r', linestyle = '-')
plt.ylim(-2000,2250)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Spannung PCC - Simulink')
plt.xlabel('Zeit t [ms]')
plt.ylabel('Spannung U [V]')
for ts_name, ts_obj  in ts_dpsim_v.items():
    if ts_name in ['Spannung_PCC_0', 'Spannung_PCC_1', 'Spannung_PCC_2']:
        plt.plot(1000*ts_obj.time, ts_obj.values, label=ts_name)
plt.axhline(y =400, color = 'r', linestyle = '-')
plt.axhline(y =-400, color = 'r', linestyle = '-')
plt.ylim(-2000,2250)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Eingangsspannung Wechselrichter - DPsim')
plt.xlabel('Zeit t [ms]')
plt.ylabel('Spannung U [V]')
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name in ['Spannung_Quelle_0','Spannung_Quelle_1','Spannung_Quelle_2']:
        plt.plot(1000*ts_obj.time, ts_obj.values, label=ts_name)
plt.axhline(y =400, color = 'r', linestyle = '-')
plt.axhline(y =-400, color = 'r', linestyle = '-')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Strom RLC - DPsim')
plt.xlabel('Zeit t [ms]')
plt.ylabel('Strom I [A]')
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name in ['Strom_RLC_0','Strom_RLC_1','Strom_RLC_2']:
        plt.plot(1000*ts_obj.time, ts_obj.values, label=ts_name)
plt.axhline(y =4120, color = 'r', linestyle = '-',label='4120 A')
plt.axhline(y =-4120, color = 'r', linestyle = '-',label='-4120 A')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Leistung PCC - DPsim')
plt.xlabel('Zeit t (ms)')
plt.ylabel('Wirkleistung P [W], Blindleistung Q [VA]')
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name in ['P_elec','Q_elec']:
        plt.plot(1000*ts_obj.time, ts_obj.values, label=ts_name)
plt.axhline(y =2465000, color = 'r', linestyle = '-', label='2.465 MW')
plt.ylim(-5e6, 6e6)
plt.legend()
plt.show()
